In [2]:
from WickContractions.wick.contract import *
from WickContractions.laph.diagram import *

from utilities import *

from IPython.display import display, Math

pprint = lambda o : display(Math(str(o)))

## Operators

At the moment we are hardcoding in the operators.  We will want to read in a dictionary of spin matrices, as well as isospin flavor combinations.

In [29]:
# TODO get input from operator code for Isospin/momentum combinations.

In [3]:
# load definition of the building blocks of the operators.
from udud_elementals import *

gammas = [('buchoff',0)]
gammas += [('A1g',i) for i in range(2)]
gammas += [('Eg',i) for i in range(2)]

gammaName = lambda irrep,i : "\\Gamma^{{ ({},{}) }}".format(irrep, i)

aOps=[]
cOps=[]
for g in gammas:
    aOps.append(Operator([baryonSink(1,['u','d','u','d'],'x_0',gammaName(g[0],g[1]))]))
    cOps.append(Operator([baryonSource(1,['d','u','d','u'], 'x_1',gammaName(g[0],g[1]))]))

## Wick Contractions

Now we run the wick contractions giving us essentially a list of tensor contractions to be performed.

In [4]:
diagrams=[]
for aOp in aOps:
    for cOp in cOps:
        for d in contract(aOp, cOp).diagrams:
            diagrams.append(d)
print("Need {} diagrams".format(len(diagrams)))

Need 100 diagrams


In [5]:
laphDiagrams = [LDiagram(d) for d in diagrams]
simplify_laphDiagrams(laphDiagrams)
#for d in laphDiagrams:
#    pprint(d)

In [6]:
for b in laphDiagrams[0].Bblocks:
    pprint(b)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
for t in laphDiagrams[0].Tblocks:
    pprint(t)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [27]:
#allBaryonTensors = find_all_baryon_tensors(laphDiagrams)
# temporary TODO make smarter version of the rest of this notebook
allBaryonTensors={}
idx=0
for g in gammas:
    allBaryonTensors['B(x_1,t_i,\\Gamma^{{ ({},{}) }})'.format(g[0],g[1])]=idx
    allBaryonTensors['B^*(x_0,t_i,t_f,\\Gamma^{{ ({},{}) }})'.format(g[0],g[1])]=idx+1
    idx+=2

allBaryonTensorsBack={v: k for k,v in allBaryonTensors.items()}

In [28]:
allBaryonTensors

{'B(x_1,t_i,\\Gamma^{ (buchoff,0) })': 0,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (buchoff,0) })': 1,
 'B(x_1,t_i,\\Gamma^{ (A1g,0) })': 2,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (A1g,0) })': 3,
 'B(x_1,t_i,\\Gamma^{ (A1g,1) })': 4,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (A1g,1) })': 5,
 'B(x_1,t_i,\\Gamma^{ (Eg,0) })': 6,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg,0) })': 7,
 'B(x_1,t_i,\\Gamma^{ (Eg,1) })': 8,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg,1) })': 9}

In [25]:
len(laphDiagrams)

100

In [31]:
def cppArrayPrint(lst):
    return str(lst).replace('[','{').replace(']','}')


def create_diagram_gpu_file(laphDiagrams, allBaryonTensors):
    print("file header")
    for d in laphDiagrams:
        for blocks,contraction in diagram_as_graph(d,allBaryonTensors).items():
            print(blocks)
            blocks=blocks.split(',')


            block0=allBaryonTensorsBack[int(blocks[0])]
            block1=allBaryonTensorsBack[int(blocks[1])]
            
            if block0[0:3]=="B^*":
                b0type='bprops'
            else: 
                b0type=''

            print("diagrams.push_back(Diagram({}[{}],{}[{}],std::vector<std::vector<int>>{}));".format(b0type,b0idx,b1type,b1idx,cppArrayPrint(contraction)))

    print("fild footer")
        

create_diagram_gpu_file(laphDiagrams,allBaryonTensors)
#for d in laphDiagrams:
    #print(diagram_as_graph(d,allBaryonTensors))
    

file header
1,0
diagrams.push_back(Diagram(what[B^*(x_0,t_i,t_f,\Gamma^{ (buchoff,0) })],what[B(x_1,t_i,\Gamma^{ (buchoff,0) })],std::vector<std::vector<int>>{{3, 0}, {2, 3}, {1, 2}, {0, 1}}));
1,0
diagrams.push_back(Diagram(what[B^*(x_0,t_i,t_f,\Gamma^{ (buchoff,0) })],what[B(x_1,t_i,\Gamma^{ (buchoff,0) })],std::vector<std::vector<int>>{{3, 2}, {2, 3}, {1, 0}, {0, 1}}));
1,0
diagrams.push_back(Diagram(what[B^*(x_0,t_i,t_f,\Gamma^{ (buchoff,0) })],what[B(x_1,t_i,\Gamma^{ (buchoff,0) })],std::vector<std::vector<int>>{{3, 0}, {2, 1}, {1, 2}, {0, 3}}));
1,0
diagrams.push_back(Diagram(what[B^*(x_0,t_i,t_f,\Gamma^{ (buchoff,0) })],what[B(x_1,t_i,\Gamma^{ (buchoff,0) })],std::vector<std::vector<int>>{{3, 2}, {2, 1}, {1, 0}, {0, 3}}));
2,1
diagrams.push_back(Diagram(what[B(x_1,t_i,\Gamma^{ (A1g,0) })],what[B^*(x_0,t_i,t_f,\Gamma^{ (buchoff,0) })],std::vector<std::vector<int>>{{3, 2}, {2, 1}, {1, 0}, {0, 3}}));
2,1
diagrams.push_back(Diagram(what[B(x_1,t_i,\Gamma^{ (A1g,0) })],what[B^*(x_0,t_

In [14]:
for d in laphDiagrams:
    pprint(d)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>